In [1]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv

from Box2D import b2World, b2PolygonShape, b2RevoluteJointDef
import pygame

In [3]:
class CustomBipedEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 60}

    def __init__(self, render_mode=None):
        super().__init__()
        self.render_mode = render_mode

        self.world = b2World(gravity=(0, -9.8))
        self.dt = 1.0 / 60.0

        # 4 continuous actions: hip_left, knee_left, hip_right, knee_right
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(4,), dtype=np.float32)

        # 14-dimensional observation
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(14,), dtype=np.float32)

        self._build_world()

        if self.render_mode == "human":
            self._init_render()

    def _build_world(self):
        # Ground
        self.ground = self.world.CreateStaticBody(
            position=(0, 0),
            shapes=b2PolygonShape(box=(50, 0.5))
        )

        # Torso
        self.torso = self.world.CreateDynamicBody(position=(0, 4))
        self.torso.CreatePolygonFixture(box=(0.4, 0.8), density=1.0, friction=0.6)

        self.upper_legs = []
        self.lower_legs = []
        self.hip_joints = []
        self.knee_joints = []

        offsets = [-0.3, 0.3]
        for offset in offsets:
            # Upper leg (thigh)
            upper = self.world.CreateDynamicBody(position=(offset, 2.5))
            upper.CreatePolygonFixture(box=(0.15, 0.7), density=1.0, friction=1.0)

            # Lower leg (shin + foot)
            lower = self.world.CreateDynamicBody(position=(offset, 0.8))
            lower.CreatePolygonFixture(box=(0.12, 0.6), density=1.0, friction=1.0)

            # Hip joint
            hip_def = b2RevoluteJointDef(
                bodyA=self.torso,
                bodyB=upper,
                localAnchorA=(offset, -0.7),
                localAnchorB=(0, 0.7),
                enableMotor=True,
                maxMotorTorque=120,  # slightly higher for DDPG
                motorSpeed=0.0
            )
            hip = self.world.CreateJoint(hip_def)

            # Knee joint
            knee_def = b2RevoluteJointDef(
                bodyA=upper,
                bodyB=lower,
                localAnchorA=(0, -0.7),
                localAnchorB=(0, 0.6),
                enableMotor=True,
                maxMotorTorque=100,
                motorSpeed=0.0,
                lowerAngle=-2.5,
                upperAngle=0.0
            )
            knee = self.world.CreateJoint(knee_def)

            self.upper_legs.append(upper)
            self.lower_legs.append(lower)
            self.hip_joints.append(hip)
            self.knee_joints.append(knee)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self.torso.position = (0, 4)
        self.torso.angle = 0
        self.torso.linearVelocity = (0, 0)
        self.torso.angularVelocity = 0

        for i, upper in enumerate(self.upper_legs):
            upper.position = self.torso.position + (-0.3 + 0.6 * i, -1.4)
            upper.angle = np.random.uniform(-0.3, 0.3)
            upper.linearVelocity = (0, 0)
            upper.angularVelocity = 0

            lower = self.lower_legs[i]
            lower.position = upper.position + (0, -1.3)
            lower.angle = upper.angle
            lower.linearVelocity = (0, 0)
            lower.angularVelocity = 0

        self.world.Step(self.dt, 6, 2)
        return self._get_obs(), {}

    def _get_obs(self):
        torso = self.torso
        ul_left, ul_right = self.upper_legs
        ll_left, ll_right = self.lower_legs

        return np.array([
            torso.position.x,
            torso.position.y - 3,             # normalized height
            torso.linearVelocity.x,
            torso.linearVelocity.y,
            torso.angle,
            torso.angularVelocity,
            ul_left.angle,
            ul_left.angularVelocity,
            ll_left.angle,
            ll_left.angularVelocity,
            ul_right.angle,
            ul_right.angularVelocity,
            ll_right.angle,
            ll_right.angularVelocity,
        ], dtype=np.float32)

    def step(self, action):
        action = np.clip(action, -1.0, 1.0)

        # Apply motor speeds (scaled)
        speeds = action * 6.0  # DDPG often benefits from larger speed scale
        for i, joint in enumerate(self.hip_joints):
            joint.motorSpeed = float(speeds[i])
        for i, joint in enumerate(self.knee_joints):
            joint.motorSpeed = float(speeds[i + 2])

        self.world.Step(self.dt, 6, 2)

        obs = self._get_obs()

        forward_vel = self.torso.linearVelocity.x
        upright = np.cos(self.torso.angle)
        energy = 0.005 * np.sum(np.square(action))

        reward = forward_vel * 2.5 + upright * 1.0 - energy

        terminated = bool(
            self.torso.position.y < 2.0 or
            abs(self.torso.angle) > np.pi / 2
        )
        if terminated:
            reward -= 100

        return obs, reward, terminated, False, {}

    def _init_render(self):
        pygame.init()
        self.screen = pygame.display.set_mode((800, 400))
        self.clock = pygame.time.Clock()

    def render(self):
        if self.render_mode != "human":
            return

        self.screen.fill((255, 255, 255))
        scale = 60

        def draw_body(body, color=(0, 0, 0)):
            for fixture in body.fixtures:
                vertices = [body.transform * v for v in fixture.shape.vertices]
                vertices = [(v[0] * scale + 400, 400 - v[1] * scale) for v in vertices]
                pygame.draw.polygon(self.screen, color, vertices)

        draw_body(self.ground, (100, 100, 100))
        draw_body(self.torso, (50, 50, 200))
        for upper in self.upper_legs:
            draw_body(upper, (200, 50, 50))
        for lower in self.lower_legs:
            draw_body(lower, (50, 200, 50))

        pygame.display.flip()
        self.clock.tick(60)

    def close(self):
        if self.render_mode == "human":
            pygame.quit()


# # ========================
# # Training with DDPG
# # ========================

# env_fn = lambda: CustomBipedEnv(render_mode=None)
# env = DummyVecEnv([env_fn])

# # Action noise for exploration
# n_actions = env.action_space.shape[0]
# action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.2 * np.ones(n_actions))

# model = DDPG(
#     "MlpPolicy",
#     env,
#     learning_rate=1e-3,           # higher LR often works well for DDPG
#     buffer_size=1_000_000,
#     learning_starts=1000,
#     batch_size=256,
#     tau=0.005,
#     gamma=0.99,
#     action_noise=action_noise,
#     train_freq=1,
#     gradient_steps=1,
#     verbose=1,
#     policy_kwargs=dict(net_arch=[256, 256]),  # larger networks help
#     device="cpu"  # explicitly CPU (remove if you have GPU)
# )

# print("Starting DDPG training...")
# model.learn(total_timesteps=350_000)  # 2M+ steps recommended for smooth walking
# model.save("ddpg_custom_biped_improved")

# env.close()
# print("Training finished and model saved!")


# ========================
# Real-time Demo
# ========================

# env = CustomBipedEnv(render_mode="human")
# model = DDPG.load("ddpg_custom_biped_improved")

# obs, _ = env.reset()
# while True:
#     for event in pygame.event.get():
#         if event.type == pygame.QUIT:
#             env.close()
#             raise SystemExit

#     action, _ = model.predict(obs, deterministic=True)
#     obs, _, terminated, _, _ = env.step(action)
#     env.render()

#     if terminated:
#         obs, _ = env.reset()

In [ ]:
# ========================
# Training with DDPG
# ========================

env_fn = lambda: CustomBipedEnv(render_mode=None)
env = DummyVecEnv([env_fn])

# Action noise for exploration
n_actions = env.action_space.shape[0]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.2 * np.ones(n_actions))

model = DDPG(
    "MlpPolicy",
    env,
    learning_rate=1e-3,           # higher LR often works well for DDPG
    buffer_size=1_000_000,
    learning_starts=1000,
    batch_size=256,
    tau=0.005,
    gamma=0.99,
    action_noise=action_noise,
    train_freq=1,
    gradient_steps=1,
    verbose=1,
    policy_kwargs=dict(net_arch=[256, 256]),  # larger networks help
    device="cpu"  # explicitly CPU (remove if you have GPU)
)

print("Starting DDPG training...")
model.learn(total_timesteps=350_000)  # 2M+ steps recommended for smooth walking
model.save("ddpg_custom_biped_improved")

env.close()
print("Training finished and model saved!")

In [4]:
# ========================
# Real-time Demo
# ========================

env = CustomBipedEnv(render_mode="human")
model = DDPG.load("ddpg_custom_biped_improved")

obs, _ = env.reset()
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            env.close()
            raise SystemExit

    action, _ = model.predict(obs, deterministic=True)
    obs, _, terminated, _, _ = env.step(action)
    env.render()

    if terminated:
        obs, _ = env.reset()

SystemExit: 

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
